In [51]:
import pandas as pd
from sqlalchemy import create_engine

def connect_to_db(user, password, host, port, database):
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
    engine = create_engine(connection_string)
    return engine

# Establish a connection to the database
user = 'root'
password = 'amelie'
host = 'localhost'
port = '3306'
database = 'sakila'

engine = connect_to_db(user, password, host, port, database)

In [52]:
# 2
def rentals_month(engine, month, year):
    query = f"""
    SELECT * FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(query, engine)
    return df

# Retrieve rental data for May and June
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)

In [53]:
# 3
def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

# Calculate the rental counts for May and June
may_rental_counts = rental_count_month(may_rentals, 5, 2005)
june_rental_counts = rental_count_month(june_rentals, 6, 2005)

In [54]:
may_rentals.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [55]:
june_rentals.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [67]:
# 4
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    combined_df['difference'] = combined_df.iloc[:, 1] - combined_df.iloc[:, 2]
    return combined_df

In [ ]:
# Compare the rental counts
comparison_df = compare_rentals(may_rental_counts, june_rental_counts)

# Display the comparison DataFrame
comparison_df.head()

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,4,0.0,6.0,-6.0
4,5,3.0,5.0,-2.0
